In [101]:
import pygame
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import time

In [102]:

def digit_recognition(big_image, small_image, symbol_classes, num_or_symbol_model, digit_recognition_model, symbol_recognition_model):
    
    new_width = 100
    new_height = 100
    resized_big_image = cv2.resize(big_image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    resized_small_image = cv2.resize(small_image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # black background, make all non black into white
    resized_big_image[resized_big_image > 0] = 255
    resized_small_image[resized_small_image > 0] = 255

    # copy of original image to use in numbers model
    copy_of_big_image = resized_big_image.copy()

    # going from black to white for first model
    inverted_big_image = 255 - resized_big_image
    inverted_small_image = 255 - resized_small_image

    reshaped_big_image = inverted_big_image.reshape(1, 100, 100, 1)
    reshaped_big_image = reshaped_big_image.astype('float32') / 255.0

    reshaped_small_image = inverted_small_image.reshape(1, 100, 100, 1)
    reshaped_small_image = reshaped_small_image.astype('float32') / 255.0

    # predict if it is a number or symbol
    num_or_symbol_index = num_or_symbol_model.predict(reshaped_big_image, verbose=0)

    num_or_symbol_prediction = 'number' if num_or_symbol_index[0][0] >= 0.5 else 'symbol'


    if num_or_symbol_prediction == 'number':
        
        number_image = copy_of_big_image.reshape(1, 100, 100, 1)
        number_image = number_image.astype('float32') / 255.0

        prediction_int = digit_recognition_model.predict(number_image, verbose=0).argmax()
        prediction = str(prediction_int)

    else:

        prediction_index = symbol_recognition_model.predict(reshaped_small_image, verbose=0).argmax()
        prediction = symbol_classes[prediction_index]

    return prediction

In [103]:
def should_merge(c1, c2):
    x1, y1, w1, h1 = cv2.boundingRect(c1)
    x2, y2, w2, h2 = cv2.boundingRect(c2)
    
    # You can adjust these thresholds as needed
    max_y_distance = 80  # Maximum allowable y-axis distance
    min_x_overlap = min(w1, w2) // 2  # Half of the minimum width for x-axis overlap
    
    y_distance = abs(y1 - y2)
    x_overlap = min(x1 + w1, x2 + w2) - max(x1, x2)
    
    return y_distance <= max_y_distance and x_overlap >= min_x_overlap

In [104]:
def get_number(equation, i, positive):

    number = ''

    i = i if positive else i+1

    while equation[i].isdigit() and i != len(equation)-1:
        
        number += equation[i]
        i += 1

    return int(number), i


In [105]:
def solving_equation(final_equation):

    answer = 0
    i = 0
    positive = True

    # for i, element in enumerate(final_equation):
    while i < len(final_equation):
        element = final_equation[i]

        if i == 0:
            # see if the element is a number
            try:
                int(element)
                number, i = get_number(final_equation, i, positive)
                answer += number

            except ValueError:
                if element == '-':
                    number, i = get_number(final_equation, i+1, positive)
                    answer -= number
                else:
                    answer = f'Starting with bad symbol: {element}'
                    break
        
        # has to iterate over a symbol here
        else:

            if element == '=':
                break
        
            # if the next element is not a number (therefore it is a symbol) and the element is a '-' then it is a negative number
            if final_equation[i+1] == '-':
                positive = False

            # if the element is a symbol
            if element == '+':
                number, i = get_number(final_equation, i+1, positive)

                if positive:
                    answer += number
                else:
                    answer += -number

            elif element == '-':
                
                number, i = get_number(final_equation, i+1, positive)
                
                if positive:
                    answer -= number
                else:
                    answer -= -number


            elif element == '*':

                number, i = get_number(final_equation, i+1, positive)
                if positive:
                    answer *= number
                else:
                    answer *= -number


            elif element == '/':
                number, i = get_number(final_equation, i+1, positive)

                if positive:
                    answer /= number
                else:
                    answer /= -number

        positive = True
            
    return answer


# print(solving_equation(['7', '8', '+', '1', '0', '=']))
# ['9', '-', '7', '-', '1', '0', '+', '1', '5', '=']


In [106]:
def draw_number(screen, equal_sign_coords, numbers, answer, erase=False):
    x, w, y, h = equal_sign_coords

    reversed_answer = str(answer)[::-1]

    digit = 0
    remaining_number = reversed_answer
    last_digit = reversed_answer[-1]
    current_position = x + w + 20 if last_digit != '-' else x + w + 20

    while remaining_number:

        differences = numbers[last_digit]

        minimum_x = np.min(np.array(differences)[:, 0])
        maximum_x = np.max(np.array(differences)[:, 0])

        starting_x = current_position + 30 + abs(minimum_x)
        starting_y = y - 20 if last_digit != '-' else y + 20

        first_erase_x = starting_x + differences[-1][0]
        first_erase_y = starting_y + differences[-1][1]

        if erase:
            pygame.draw.circle(screen, (0, 0, 0), (first_erase_x, first_erase_y), 10)


        for i, diff in enumerate(differences):
                
            current_drawing_x = starting_x + diff[0]
            current_drawing_y = starting_y + diff[1]

            if i < len(differences)-1 and i > 0:
                # x and y coords when drawing
                next_drawing_x = starting_x + differences[i+1][0]
                next_drawing_y = starting_y + differences[i+1][1]

                # x and y coords when erasing
                erase_x = starting_x + differences[len(differences)-i-1][0]
                erase_y = starting_y + differences[len(differences)-i-1][1]

                next_erase_x = starting_x + differences[len(differences)-i-2][0]
                next_erase_y = starting_y + differences[len(differences)-i-2][1]

                if not erase:
                    pygame.draw.circle(screen, (255, 165, 0), (current_drawing_x, current_drawing_y), 10)
                    pygame.draw.circle(screen, (255, 0, 0), (next_drawing_x, next_drawing_y), 10)
                else:
                    pygame.draw.circle(screen, (0, 0, 0), (erase_x, erase_y), 10)
                    pygame.draw.circle(screen, (255, 0, 0), (next_erase_x, next_erase_y), 10)

                pygame.display.flip()

                # drawing numbers
                if not erase:
                    if last_digit == '0' or last_digit == '2':
                        time.sleep(0.002)
                    else:
                        time.sleep(0.004)

                # erasing numbers
                else:
                    time.sleep(0.002)

        last_drawing_x = starting_x + differences[-1][0]
        last_drawing_y = starting_y + differences[-1][1]

        last_erase_x = starting_x + differences[0][0]
        last_erase_y = starting_y + differences[0][1]
        

        if not erase:
            pygame.draw.circle(screen, (255, 165, 0), (last_drawing_x, last_drawing_y), 10)
        else:
            pygame.draw.circle(screen, (0, 0, 0), (last_erase_x, last_erase_y), 10)


        pygame.display.flip()

        digit += 1
        remaining_number = remaining_number[:-1]
        
        if remaining_number:
            last_digit = remaining_number[-1]

        current_position = starting_x + maximum_x

In [107]:
def preprocess_image(gray_image, x, w, y, h): 
    digit_roi = gray_image[y:y + h, x:x + w]

    aspect_ratio = w / h
    if aspect_ratio > 1:  # Width is greater than height
        big_w = 60
        big_h = int(60 / aspect_ratio)

        small_w = 30
        small_h = int(30 / aspect_ratio)

    else:  # Height is greater than or equal to width
        big_w = int(60 * aspect_ratio)
        big_h = 60

        small_w = int(30 * aspect_ratio)
        small_h = 30

    big_resized_digit = cv2.resize(digit_roi, (big_w, big_h))
    big_canvas = np.zeros((100, 100), dtype='uint8')
    big_start_x = (big_canvas.shape[1] - big_w) // 2
    big_start_y = (big_canvas.shape[0] - big_h) // 2
    big_canvas[big_start_y:big_start_y + big_h, big_start_x:big_start_x + big_w] = big_resized_digit

    small_resized_digit = cv2.resize(digit_roi, (small_w, small_h))
    small_canvas = np.zeros((100, 100), dtype='uint8')
    small_start_x = (small_canvas.shape[1] - small_w) // 2
    small_start_y = (small_canvas.shape[0] - small_h) // 2
    small_canvas[small_start_y:small_start_y + small_h, small_start_x:small_start_x + small_w] = small_resized_digit

    return big_canvas, small_canvas


In [108]:
def find_merged_contours(screen, screen_height, reset_button_height, final_equation, erase_mode):
    pixel_array = pygame.surfarray.array3d(screen)

                                 # subtract 3*h because you do not count the bottom rows in the image detection
    pixel_array = pixel_array[:, :screen_height - 3*reset_button_height, :]

    pixel_array = np.transpose(pixel_array, (1, 0, 2))

    gray_image = cv2.cvtColor(pixel_array, cv2.COLOR_BGR2GRAY)

    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

    merged_contours = []
    visited = [False] * len(contours)

    

    for i in range(len(contours)):
        
        if not visited[i]:
            merged_contour = contours[i].copy()
            visited[i] = True
            
            for j in range(i + 1, len(contours)):
                if not visited[j] and should_merge(contours[i], contours[j]):
                    if j == len(contours)-1:
                        if not erase_mode:
                            del final_equation[-1]

                    merged_contour = np.concatenate((merged_contour, contours[j]))
                    visited[j] = True
            
            merged_contours.append(merged_contour)

    return merged_contours, gray_image

In [109]:
def find_contours(screen, classes, final_equation, num_or_symbol_model, digit_recognition_model, symbol_recognition_model, screen_height, reset_button_height, leftmost_x_coord, rightmost_x_coord, rescan=False):

    merged_contours, gray_image = find_merged_contours(screen, screen_height, reset_button_height, final_equation, False)

    if merged_contours:
        right_x, right_y, right_w, right_h = cv2.boundingRect(merged_contours[-1])
        left_x, left_y, left_w, left_h = cv2.boundingRect(merged_contours[0])

        
        # when you add to the equation from the left
        if left_x < leftmost_x_coord: 
            leftmost_x_coord = left_x

            big_canvas, small_canvas = preprocess_image(gray_image, left_x, left_w, left_y, left_h)

            prediction = digit_recognition(big_canvas, small_canvas, classes, num_or_symbol_model, digit_recognition_model, symbol_recognition_model)
            final_equation.insert(0, prediction)

            # pause to add a delay between now and the drawing of the solution
            if len(merged_contours) > 1:
                time.sleep(1)

        # if somebody writes in the middle of the equation
        elif (left_x == leftmost_x_coord and right_x == rightmost_x_coord) or rescan:
            # rescan the equation
            final_equation = []
            for contour in merged_contours:
                current_x, current_y, current_w, current_h = cv2.boundingRect(contour)
                big_canvas, small_canvas = preprocess_image(gray_image, current_x, current_w, current_y, current_h)
                prediction = digit_recognition(big_canvas, small_canvas, classes, num_or_symbol_model, digit_recognition_model, symbol_recognition_model)
                final_equation.append(prediction)            
            
        # when you write from left to right
        else:
            rightmost_x_coord = right_x

            big_canvas, small_canvas = preprocess_image(gray_image, right_x, right_w, right_y, right_h)

            prediction = digit_recognition(big_canvas, small_canvas, classes, num_or_symbol_model, digit_recognition_model, symbol_recognition_model)
            final_equation.append(prediction)
            
            if prediction == '+' or (prediction == '*' and final_equation[-2] == '*'):
                del final_equation[-2]

        print(final_equation)

        return gray_image, final_equation, (right_x, right_w, right_y, right_h), leftmost_x_coord, rightmost_x_coord

In [110]:
def draw_reset_button(screen, position, text):
    font = pygame.font.SysFont('Arial', 40)
    text_render = font.render(text, True, (255, 255, 255))
    x, y, w, h = text_render.get_rect()
    x, y = position
    
    return screen.blit(text_render, (x, y))

In [111]:
def draw_circle(screen, radius, circle_location, color, erase_mode):

    pygame.draw.circle(screen, color, circle_location, radius)
    
    # drawing mode icon
    if not erase_mode:
        image = pygame.image.load('erase_icon.jpg')

    # erase mode icon
    else:
        image = pygame.image.load('erase_icon_green.jpg')

    image = pygame.transform.scale(image, (40, 40))
    
    image_rect = image.get_rect()
    image_rect.center = circle_location
    
    return screen.blit(image, image_rect)

In [112]:
def get_text_width_and_height(text):
    font = pygame.font.SysFont('Arial', 40)
    text_render = font.render(text, True, (255, 255, 255))
    x, y, w , h = text_render.get_rect()
    
    return w, h

In [113]:
def animate_solution(screen, final_equation, answer, numbers, screen_height, reset_button_height, equal_sign_coords, equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h, rightmost_x_coord):
    running = True
    if final_equation[-1] != '=' and '=' in final_equation:
        try:
            draw_number(screen, (equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h), numbers, answer, True)
            
            equal_sign_index = final_equation.index('=')
            final_equation = final_equation[:equal_sign_index+1]
            answer = solving_equation(final_equation)
            print(f'answer: {answer}')

            if not isinstance(answer, int):
                answer = int(answer)

            draw_number(screen, (equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h), numbers, answer)

            reversed_answer = str(answer)[::-1]
            while reversed_answer:
                final_equation.append(reversed_answer[-1])
                reversed_answer = reversed_answer[:-1]

            # update the rightmost x point after drawing the numbers
            merged_contours, gray_image = find_merged_contours(screen, screen_height, reset_button_height, final_equation, False)
            x, y, w, h = cv2.boundingRect(merged_contours[-1])
            rightmost_x_coord = x

        except Exception as e:
            print(f'Failed to resolve the equation: {final_equation}')
            print(e)

        
    elif final_equation[-1] == '=':
        equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h = equal_sign_coords
        
        try:
            answer = solving_equation(final_equation)

        # equation failed to solve
        except Exception as e:
            print()
            print('Error')
            print('Invalid Equation')
            print('Try Again')
            running = False
            

        # equation started with an invalid symbol
        if final_equation[0] == '*' or final_equation[0] == '+' or final_equation[0] == '/' or final_equation[0] == '=':   
            print()
            print(f'Error: Equation started with a \'{final_equation[0]}\'')
            running = False
            

        # when the answer is valid
        else:
            print(f'answer: {answer}')

        
        time.sleep(1)
        
        # to turn potential floats into integers
        if not isinstance(answer, int):
            answer = int(answer)

        # append the solution to the final equation
        reversed_answer = str(answer)[::-1]
        while reversed_answer:
            final_equation.append(reversed_answer[-1])
            reversed_answer = reversed_answer[:-1]
            
        print(final_equation)
        
        draw_number(screen, equal_sign_coords, numbers, answer)
                    
        # update the rightmost x point after drawing the numbers
        merged_contours, gray_image = find_merged_contours(screen, screen_height, reset_button_height, final_equation, False)
        x, y, w, h = cv2.boundingRect(merged_contours[-1])
        rightmost_x_coord = x
            
    return running, answer, equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h, rightmost_x_coord


In [116]:

pygame.init()

screen_width = 800
screen_height = 600

screen_width = 1500
screen_height = 700

screen_width = 1550
screen_height = 960

black = (0, 0, 0)
orange = (255, 165, 0)
red = (255, 0, 0)
white = (255, 255, 255)
green = (0, 255, 0)

num_or_symbol_model = load_model('num_or_symbol.h5')
digit_recognition_model = load_model('digit_recognizer_model.h5')
symbol_recognition_model = load_model('symbol_recognition_model.h5')

symbol_classes = {
    0: '+',
    1: '.',
    2: '/',
    3: '=',
    4: '*',
    5: '-',
    6: 'x',
    7: 'y',
    8: 'z',
}

zero_differences = [(-1, 1), (-4, 5), (-4, 5), (-7, 10), (-7, 10), (-8, 13), (-8, 13), (-10, 17), (-10, 17), (-12, 22), (-12, 22), (-14, 29), (-14, 29), (-15, 33), (-16, 40), (-16, 40), (-17, 47), (-17, 47), (-17, 56), (-17, 56), (-17, 61), (-17, 61), (-17, 69), (-17, 69), (-16, 79), (-16, 79), (-16, 82), (-15, 90), (-15, 90), (-13, 96), (-13, 96), (-12, 102), (-12, 102), (-10, 108), (-10, 108), (-8, 112), (-8, 112), (-6, 117), (-6, 117), (-4, 120), (-4, 120), (-2, 123), (0, 126), (0, 126), (2, 127), (2, 127), (4, 131), (4, 131), (5, 132), (8, 135), (8, 135), (12, 137), (12, 137), (16, 138), (16, 138), (19, 139), (19, 139), (20, 140), (24, 140), (24, 140), (27, 139), (27, 139), (32, 138), (32, 138), (38, 136), (38, 136), (43, 133), (43, 133), (46, 132), (46, 132), (48, 129), (52, 126), (52, 126), (56, 122), (56, 122), (59, 118), (59, 118), (61, 112), (61, 112), (64, 107), (64, 107), (67, 100), (67, 100), (69, 92), (69, 92), (71, 84), (71, 84), (71, 78), (71, 78), (72, 71), (72, 71), (71, 68), (70, 58), (70, 58), (69, 52), (69, 52), (68, 45), (68, 45), (67, 40), (67, 40), (64, 34), (64, 34), (63, 31), (61, 22), (61, 22), (61, 22), (60, 20), (58, 15), (58, 15), (56, 11), (56, 11), (53, 8), (52, 5), (50, 3), (50, 3), (48, 0), (48, 0), (45, -2), (45, -2), (43, -4), (43, -4), (40, -5), (40, -5), (39, -6), (36, -8), (33, -8), (33, -8), (30, -9), (30, -9), (27, -9), (27, -9), (25, -9), (23, -9), (23, -9), (20, -9), (20, -9), (18, -9), (18, -9), (16, -8), (16, -8), (11, -6), (11, -6), (7, 0), (7, 0), (4, 7), (4, 7), (3, 9), (1, 13)]
one_differences = [(1, 1), (2, 5), (2, 5), (3, 10), (3, 10), (4, 14), (4, 14), (5, 21), (5, 21), (5, 25), (6, 32), (6, 32), (6, 41), (6, 41), (7, 49), (7, 49), (7, 59), (7, 59), (8, 67), (8, 67), (8, 71), (9, 79), (9, 79), (9, 86), (9, 86), (10, 93), (10, 93), (10, 100), (10, 100), (10, 103), (10, 109), (10, 109), (10, 114), (10, 114), (10, 119), (10, 119), (11, 125), (11, 125), (11, 127), (11, 129), (10, 133), (10, 133), (10, 135), (10, 135), (10, 138), (10, 138), (10, 141), (10, 142), (10, 143), (10, 144), (10, 145)]
two_differences = [(0, -1), (4, -2), (4, -2), (8, -2), (8, -2), (11, -2), (11, -2), (13, -1), (13, -1), (15, -1), (20, 1), (20, 1), (24, 3), (24, 3), (29, 6), (29, 6), (34, 9), (34, 9), (39, 14), (39, 14), (44, 20), (44, 20), (46, 23), (50, 31), (50, 31), (53, 38), (53, 38), (56, 46), (56, 46), (58, 53), (58, 53), (60, 61), (60, 61), (60, 65), (61, 73), (61, 73), (61, 79), (61, 79), (61, 86), (61, 86), (61, 93), (61, 93), (60, 98), (60, 98), (60, 100), (59, 105), (59, 105), (57, 111), (57, 111), (56, 115), (56, 115), (55, 117), (52, 122), (52, 122), (50, 124), (50, 124), (47, 127), (47, 127), (44, 130), (44, 130), (39, 132), (39, 132), (36, 134), (32, 136), (32, 136), (30, 136), (24, 137), (24, 137), (20, 137), (20, 137), (15, 136), (15, 136), (12, 134), (12, 134), (8, 131), (8, 131), (6, 131), (4, 128), (4, 128), (4, 126), (4, 126), (3, 125), (2, 121), (2, 121), (2, 117), (2, 117), (3, 115), (3, 115), (6, 109), (6, 109), (8, 107), (12, 105), (12, 105), (16, 102), (16, 102), (21, 99), (21, 99), (28, 98), (28, 98), (35, 98), (35, 98), (39, 99), (39, 99), (42, 99), (48, 100), (48, 100), (55, 103), (55, 103), (60, 106), (60, 106), (66, 109), (66, 109), (70, 113), (70, 113), (72, 114), (76, 117), (76, 117), (81, 121), (81, 121), (85, 123), (85, 123), (88, 125), (88, 125), (90, 126), (92, 129), (92, 129), (94, 130), (95, 131), (96, 131), (97, 132), (97, 132), (99, 132), (100, 134)]
three_differences = [(2, -2), (5, -4), (5, -4), (10, -8), (10, -8), (14, -9), (14, -9), (15, -9), (19, -9), (19, -9), (24, -9), (24, -9), (29, -8), (29, -8), (34, -5), (34, -5), (36, -3), (40, 1), (40, 1), (44, 6), (44, 6), (47, 10), (47, 10), (48, 17), (48, 17), (49, 24), (49, 24), (49, 27), (47, 33), (47, 33), (46, 38), (46, 38), (43, 42), (43, 42), (39, 47), (39, 47), (36, 51), (36, 51), (34, 53), (30, 57), (30, 57), (27, 59), (27, 59), (27, 60), (25, 61), (24, 61), (23, 62), (22, 63), (22, 63), (21, 64), (21, 65), (20, 65), (22, 66), (22, 66), (26, 69), (26, 69), (33, 73), (33, 73), (41, 76), (41, 76), (47, 78), (47, 78), (50, 79), (55, 82), (55, 82), (59, 85), (59, 85), (61, 89), (61, 89), (63, 97), (63, 97), (63, 101), (62, 107), (62, 107), (60, 114), (60, 114), (58, 119), (58, 119), (54, 125), (54, 125), (51, 128), (51, 128), (49, 131), (46, 132), (46, 132), (43, 136), (43, 136), (39, 138), (39, 138), (35, 140), (35, 140), (33, 141), (31, 141), (27, 142), (27, 142), (25, 142), (25, 142), (22, 142), (22, 142), (21, 142), (19, 142), (19, 142), (17, 142), (17, 142), (16, 142), (15, 142)]
four_differences = [(1, 6), (1, 6), (1, 11), (1, 11), (2, 17), (2, 17), (3, 25), (3, 25), (4, 34), (4, 34), (4, 38), (5, 46), (5, 46), (6, 55), (6, 55), (6, 63), (6, 63), (7, 67), (7, 67), (7, 72), (7, 72), (7, 74), (7, 74), (7, 75), (7, 78), (7, 78), (8, 78), (8, 79), (8, 80), (8, 81), (10, 82), (10, 82), (12, 82), (12, 82), (18, 83), (18, 83), (24, 84), (24, 84), (28, 84), (28, 84), (33, 83), (33, 83), (36, 83), (41, 82), (41, 82), (43, 82), (43, 82), (44, 82), (46, 82), (46, 82), (47, 82), (48, 82), (50, 82), (50, 82), (52, 82), (52, 82), (56, 1), (56, 4), (57, 10), (57, 10), (58, 16), (58, 16), (59, 25), (59, 25), (60, 33), (60, 33), (60, 41), (60, 41), (61, 46), (62, 54), (62, 54), (63, 62), (63, 62), (64, 71), (64, 71), (65, 79), (65, 79), (66, 86), (66, 86), (67, 89), (68, 95), (68, 95), (68, 102), (68, 102), (68, 108), (68, 108), (69, 114), (69, 114), (69, 116), (70, 122), (70, 122), (70, 126), (70, 126), (71, 128), (71, 132), (71, 132), (71, 134), (71, 134), (71, 135), (71, 137), (71, 139), (71, 139), (71, 140), (71, 141), (72, 143), (72, 143), (72, 144), (72, 145), (72, 146), (72, 147), (72, 148), (72, 149)]
five_differences = [(0, 5), (0, 5), (0, 7), (-1, 11), (-1, 11), (-1, 16), (-1, 16), (0, 22), (0, 22), (0, 31), (0, 31), (0, 39), (0, 39), (1, 48), (1, 48), (2, 52), (3, 56), (3, 56), (3, 60), (3, 60), (4, 62), (4, 63), (4, 64), (4, 66), (4, 66), (4, 68), (4, 68), (5, 68), (6, 68), (8, 68), (8, 68), (12, 67), (20, 65), (20, 65), (29, 63), (29, 63), (36, 62), (36, 62), (45, 63), (45, 63), (52, 66), (52, 66), (56, 68), (66, 75), (66, 75), (73, 82), (73, 82), (80, 90), (80, 90), (84, 96), (84, 96), (86, 104), (86, 104), (86, 108), (86, 108), (86, 112), (84, 116), (84, 116), (81, 121), (81, 121), (73, 128), (73, 128), (65, 133), (65, 133), (56, 135), (56, 135), (52, 135), (44, 135), (44, 135), (39, 134), (39, 134), (31, 134), (31, 134), (28, 133), (28, 133), (25, 132), (25, 132), (24, 132), (22, 132), (22, 132), (21, 131), (21, 131), (20, 131), (0, -2), (4, -3), (4, -3), (6, -3), (12, -4), (12, -4), (16, -4), (16, -4), (20, -5), (20, -5), (25, -6), (25, -6), (28, -6), (31, -7), (36, -7), (36, -7), (40, -8), (40, -8), (44, -8), (48, -8), (48, -8), (51, -8), (51, -8), (52, -8), (54, -8), (54, -8), (56, -8), (56, -8), (57, -9)]
six_differences = [(-4, 2), (-4, 2), (-7, 3), (-7, 3), (-10, 6), (-10, 6), (-12, 8), (-12, 8), (-15, 11), (-15, 11), (-17, 13), (-21, 17), (-21, 17), (-24, 21), (-24, 21), (-28, 28), (-28, 28), (-31, 34), (-31, 34), (-32, 37), (-35, 43), (-35, 43), (-36, 49), (-36, 49), (-39, 55), (-39, 55), (-40, 62), (-40, 62), (-41, 69), (-41, 69), (-41, 72), (-42, 79), (-42, 79), (-43, 86), (-43, 86), (-44, 93), (-44, 93), (-44, 99), (-44, 99), (-44, 103), (-44, 109), (-44, 109), (-43, 115), (-43, 115), (-43, 121), (-43, 121), (-42, 126), (-42, 126), (-40, 131), (-40, 131), (-39, 136), (-39, 136), (-37, 139), (-34, 143), (-34, 143), (-31, 147), (-31, 147), (-27, 152), (-27, 152), (-24, 154), (-18, 158), (-18, 158), (-18, 158), (-14, 160), (-8, 163), (-8, 163), (-1, 165), (-1, 165), (4, 165), (4, 165), (11, 165), (11, 165), (14, 164), (19, 163), (19, 163), (24, 160), (24, 160), (27, 158), (27, 158), (29, 155), (29, 155), (32, 151), (32, 151), (33, 147), (33, 147), (34, 145), (34, 143), (34, 141), (33, 139), (33, 137), (29, 131), (29, 131), (28, 128), (28, 128), (23, 124), (23, 124), (19, 121), (19, 121), (16, 121), (16, 121), (14, 120), (11, 117), (11, 117), (7, 116), (7, 116), (4, 115), (4, 115), (0, 115), (0, 115), (-4, 115), (-4, 115), (-6, 115), (-8, 115), (-12, 116), (-12, 116), (-16, 117), (-16, 117), (-17, 119), (-17, 119), (-19, 120), (-19, 120), (-20, 121), (-21, 121), (-23, 121), (-24, 122), (-25, 122), (-26, 124), (-27, 125), (-28, 126), (-28, 127), (-29, 127), (-29, 128)]
seven_differences = [(3, 0), (6, 1), (6, 1), (9, 1), (9, 1), (12, 1), (12, 1), (17, 1), (17, 1), (23, 2), (23, 2), (25, 2), (30, 3), (30, 3), (36, 3), (36, 3), (41, 4), (41, 4), (47, 4), (47, 4), (52, 4), (52, 4), (56, 5), (56, 5), (61, 5), (61, 5), (64, 5), (68, 5), (68, 5), (70, 5), (70, 5), (73, 6), (73, 6), (76, 6), (76, 6), (78, 7), (78, 7), (80, 7), (81, 8), (84, 8), (85, 8), (87, 9), (87, 9), (88, 9), (89, 9), (90, 9), (91, 10), (91, 10), (90, 12), (90, 12), (90, 13), (90, 17), (90, 17), (88, 25), (88, 25), (84, 34), (84, 34), (80, 44), (80, 44), (79, 48), (79, 48), (76, 54), (76, 54), (74, 58), (72, 65), (72, 65), (69, 71), (69, 71), (66, 77), (66, 77), (65, 80), (63, 85), (63, 85), (60, 93), (60, 93), (59, 99), (59, 99), (57, 103), (57, 103), (56, 109), (56, 109), (55, 111), (53, 114), (53, 114), (52, 118), (52, 118), (51, 119), (51, 119), (49, 123), (48, 126), (48, 126), (48, 127), (47, 130), (47, 130), (46, 133), (46, 133), (45, 136), (45, 136), (44, 137), (44, 138), (44, 140), (43, 142), (43, 142), (43, 144), (43, 144), (42, 145), (42, 145), (41, 145), (41, 146), (41, 147), (40, 148), (40, 148), (40, 149), (40, 150)]
eight_differences = [(-4, 1), (-4, 1), (-8, 1), (-8, 1), (-11, 2), (-11, 2), (-13, 2), (-13, 2), (-15, 3), (-15, 3), (-17, 3), (-17, 3), (-20, 4), (-20, 4), (-22, 5), (-24, 7), (-24, 7), (-28, 9), (-28, 9), (-31, 12), (-31, 12), (-34, 15), (-34, 15), (-35, 16), (-38, 23), (-38, 23), (-40, 27), (-40, 27), (-41, 33), (-41, 33), (-42, 39), (-42, 39), (-42, 44), (-42, 44), (-41, 50), (-41, 50), (-40, 53), (-40, 53), (-40, 54), (-37, 59), (-37, 59), (-33, 63), (-33, 63), (-27, 67), (-27, 67), (-21, 69), (-21, 69), (-18, 70), (-10, 72), (-10, 72), (-4, 75), (-4, 75), (2, 77), (2, 77), (8, 79), (8, 79), (15, 83), (15, 83), (18, 84), (24, 87), (24, 87), (27, 91), (27, 91), (31, 95), (31, 95), (34, 99), (34, 99), (36, 103), (39, 109), (39, 109), (40, 114), (40, 114), (40, 119), (40, 119), (40, 123), (40, 123), (40, 127), (39, 130), (39, 130), (36, 135), (36, 135), (33, 138), (33, 138), (29, 142), (29, 142), (25, 144), (25, 144), (20, 146), (20, 146), (18, 146), (13, 147), (13, 147), (8, 147), (8, 147), (2, 146), (2, 146), (-3, 144), (-3, 144), (-8, 143), (-8, 143), (-11, 140), (-11, 140), (-12, 140), (-13, 138), (-16, 137), (-16, 137), (-18, 135), (-20, 132), (-20, 132), (-21, 129), (-21, 129), (-22, 127), (-22, 121), (-22, 121), (-20, 115), (-20, 115), (-18, 109), (-18, 109), (-15, 103), (-15, 103), (-13, 100), (-9, 95), (-9, 95), (-6, 91), (-6, 91), (-3, 86), (-3, 86), (1, 81), (1, 81), (4, 77), (4, 77), (6, 75), (9, 70), (9, 70), (12, 65), (12, 65), (16, 59), (16, 59), (18, 54), (18, 54), (20, 49), (20, 49), (21, 47), (22, 42), (22, 42), (23, 39), (23, 39), (23, 33), (23, 33), (22, 28), (22, 28), (21, 23), (21, 23), (20, 23), (20, 21), (19, 19), (19, 19), (18, 18), (17, 15), (17, 15), (16, 14), (16, 14), (15, 13), (13, 11), (13, 11), (12, 11), (11, 9), (11, 9), (10, 8), (10, 8), (9, 8), (8, 7)]
nine_differences = [(-4, 0), (-4, 0), (-6, 1), (-10, 1), (-10, 1), (-14, 3), (-14, 3), (-18, 5), (-18, 5), (-23, 6), (-23, 6), (-25, 8), (-30, 10), (-30, 10), (-34, 14), (-34, 14), (-38, 17), (-38, 17), (-42, 21), (-42, 21), (-45, 25), (-45, 25), (-46, 27), (-49, 32), (-49, 32), (-50, 38), (-50, 38), (-51, 43), (-51, 43), (-52, 48), (-52, 48), (-51, 53), (-51, 53), (-51, 54), (-50, 59), (-50, 59), (-49, 62), (-49, 62), (-47, 65), (-47, 65), (-44, 69), (-44, 69), (-41, 71), (-41, 71), (-36, 74), (-36, 74), (-33, 74), (-29, 75), (-29, 75), (-23, 75), (-23, 75), (-18, 74), (-18, 74), (-13, 73), (-13, 73), (-9, 71), (-9, 71), (-4, 69), (-4, 69), (-2, 68), (-2, 68), (-1, 66), (3, 63), (3, 63), (4, 61), (4, 61), (7, 57), (7, 57), (7, 55), (9, 52), (10, 48), (10, 48), (11, 42), (11, 42), (11, 38), (10, 34), (10, 34), (10, 32), (9, 30), (9, 29), (8, 26), (8, 26), (7, 24), (7, 22), (7, 21), (6, 19), (6, 19), (6, 18), (5, 18), (5, 16), (5, 16), (5, 15), (4, 14), (4, 13), (4, 12), (4, 13), (5, 14), (5, 16), (6, 22), (6, 22), (7, 30), (7, 30), (8, 37), (8, 37), (9, 43), (9, 43), (10, 49), (10, 49), (10, 52), (10, 57), (10, 57), (11, 64), (11, 64), (11, 70), (11, 70), (11, 75), (11, 75), (11, 78), (12, 84), (12, 84), (12, 90), (12, 90), (13, 95), (13, 95), (13, 102), (13, 102), (13, 104), (14, 109), (14, 109), (14, 115), (14, 115), (15, 121), (15, 121), (15, 126), (15, 126), (15, 131), (15, 131), (15, 135), (15, 135), (15, 138), (15, 140), (15, 140), (15, 143), (15, 143), (15, 145), (15, 146), (15, 148), (15, 150), (15, 153), (15, 153), (15, 154), (15, 157), (15, 157), (15, 158), (15, 160), (15, 160), (15, 161), (15, 162), (15, 163), (15, 164)]
negative_sign_differences = [(4, 1), (4, 1), (8, 1), (8, 1), (12, 1), (12, 1), (17, 2), (17, 2), (20, 2), (24, 3), (24, 3), (29, 3), (29, 3), (34, 3), (34, 3), (40, 3), (40, 3), (42, 3), (48, 4), (48, 4), (48, 4), (49, 4), (52, 4), (52, 4), (54, 4), (56, 4), (58, 4), (58, 4), (61, 4), (61, 4), (64, 4), (64, 4), (65, 4), (67, 4), (67, 4), (68, 4), (69, 3)]

numbers = {'0': zero_differences,
           '1': one_differences,
           '2': two_differences,
           '3': three_differences,
           '4': four_differences,
           '5': five_differences,
           '6': six_differences,
           '7': seven_differences,
           '8': eight_differences,
           '9': nine_differences,
           '-': negative_sign_differences}

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('My Math Notes')

button_width, reset_button_height = get_text_width_and_height('Reset')
erase_button_radius = 30


screen.fill(black)


running = True
drawing = False
red_stripe_index = 0
prediction = 0

leftmost_x_coord = screen_width+1
rightmost_x_coord = 0
num_of_contours = 0

clock = pygame.time.Clock()

final_equation = []
answer = 0
gray_image = None

erase_mode = False
first_detection_after_erase_mode = False
erase_button_color = white

equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h = 0, 0, 0, 0

while running:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # when a click is held down (includes when your finger is moving on the screen when using the touchpad)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mouse_pos = event.pos
            
            if not reset_button.collidepoint(mouse_pos) and not erase_button.collidepoint(mouse_pos) and mouse_pos[1] < screen_height - 3*reset_button_height:
                drawing = True

        # when a click is released (when you pick your finger up from the screen)
        elif event.type == pygame.MOUSEBUTTONUP:
            mouse_pos = event.pos
            
            # clicked on the reset button
            if reset_button.collidepoint(mouse_pos):
                print('Reset!')
                leftmost_x_coord = screen_width + 1
                final_equation = []
                screen.fill(black)
            
            # clicked on the erase mode button
            elif erase_button.collidepoint(mouse_pos):
                erase_mode = not erase_mode

                if erase_mode:
                    erase_button_color = green
                else:
                    erase_button_color = white
                    gray_image, final_equation, equal_sign_coords, leftmost_x_coord, rightmost_x_coord = find_contours(screen, symbol_classes, final_equation, num_or_symbol_model, digit_recognition_model, symbol_recognition_model, screen_height, reset_button_height, leftmost_x_coord, rightmost_x_coord, True)
                    print(final_equation)
                
                print('Erasing' if erase_mode else 'Writing')

            elif mouse_pos[1] > screen_height - 3*reset_button_height:
                continue
            
            # release your mouse or finger on a non button
            else:
                drawing = False

                # finds the contours and the final equation
                if not erase_mode:
                    gray_image, final_equation, equal_sign_coords, leftmost_x_coord, rightmost_x_coord = find_contours(screen, symbol_classes, final_equation, num_or_symbol_model, digit_recognition_model, symbol_recognition_model, screen_height, reset_button_height, leftmost_x_coord, rightmost_x_coord)
                
                # draws the answer onto the screen
                if final_equation and not erase_mode:
                    running, answer, equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h, rightmost_x_coord = animate_solution(screen, final_equation, answer, numbers, screen_height, reset_button_height, equal_sign_coords, equal_sign_x, equal_sign_w, equal_sign_y, equal_sign_h, rightmost_x_coord)
                    

                    
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q or event.key == pygame.K_RETURN:
                running = False

    if drawing:
        if not erase_mode:
            mouse_position = pygame.mouse.get_pos()
            pygame.draw.circle(screen, orange, mouse_position, 10)
        else:
            mouse_position = pygame.mouse.get_pos()
            pygame.draw.circle(screen, black, mouse_position, 25)

    reset_button = draw_reset_button(screen, (screen_width//2 - (button_width//2), screen_height - 2*reset_button_height - 30), 'Reset')
    erase_button = draw_circle(screen, 30, (int(screen_width*0.9), int(screen_height*0.9)), erase_button_color, erase_mode)

    pygame.display.flip()


pygame.quit()
sys.exit()



['1']
['1', '1']
['1', '+']
['1', '+', '1']
['1', '+', '1', '-']
['1', '+', '1', '=']
answer: 2
['1', '+', '1', '=', '2']
Reset!
['3']
['3', '*']
['3', '*']
['3', '*', '2']
['3', '*', '2', '6']
['3', '*', '2', '6', '9']
['3', '*', '2', '6', '/']
['3', '*', '2', '6', '*']
['3', '*', '2', '6', '*', '1']
['3', '*', '2', '6', '*', '1', '-']
['3', '*', '2', '6', '*', '1', '=']
answer: 78
['3', '*', '2', '6', '*', '1', '=', '7', '8']
Reset!
['1']
['1', '1']
['1', '+']
['1', '+', '1']
['1', '+', '1', '-']
['1', '+', '1', '=']
answer: 2
['1', '+', '1', '=', '2']
Reset!
['3']
['3', '*']
['3', '*']
['3', '*', '2']
['3', '*', '2', '6']
['3', '*', '2', '6', '9']
['3', '*', '2', '6', '-']
['3', '*', '2', '6', '/']
['3', '*', '2', '6', '/', '1']
Reset!
['1']
['1', '1']
['1', '+']
['1', '+', '1']
['1', '+', '1', '-']
['1', '+', '1', '=']
answer: 2
['1', '+', '1', '=', '2']
Reset!
['3']
['3', '*']
['3', '*']
['3', '*', '2']
['3', '*', '2', '6']
['3', '*', '2', '6', '9']
['3', '*', '2', '+']
['3', '*',

SystemExit: 